In [74]:
import pandas as pd
import snscrape.modules.twitter as sntwitter
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import text2emotion as te
import emoji
from collections import Counter
import matplotlib.pyplot as plt

In [75]:
tweets = []

for i, tweet in enumerate(sntwitter.TwitterSearchScraper('#BlackLivesMatter since:2022-05-22 until:2023-05-22').get_items()):
    if i+1 > 1000:
        break
    tweets.append([tweet.id, tweet.date, tweet.user.username, tweet.rawContent, tweet.user.location, tweet.likeCount])

df = pd.DataFrame(tweets, columns=['id', 'date', 'username', 'content', 'location', 'likes'])
print(df.shape)

df.to_csv('tweets.csv', index=False)

Unavailable user in card on tweet 1660392261723496448
User 14204245 not found in user refs in card on tweet 1660392261723496448
Unavailable user in card on tweet 1660377399660355587
User 14204245 not found in user refs in card on tweet 1660377399660355587


(1000, 6)


In [76]:
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    # Deleting links
    text = re.sub(r'http\S+|www.\S+', '', text, flags=re.MULTILINE)
    # Deleting hashtags and mentions
    text = re.sub(r'#\S+|@\S+', '', text)
    # # Deleting 
    # text = re.sub(r'\W', ' ', text)
    # Tokenizing and lemmatizing
    text = word_tokenize(text)
    text = [lemmatizer.lemmatize(word) for word in text if word not in stop_words]
    return text

df['processed_text'] = df['content'].apply(preprocess_text)

In [77]:
nltk.download('vader_lexicon')

def emotion_analysis(text):
    emotions = te.get_emotion(' '.join(text))
    return emotions

def sentiment_analysis(text):
    sid = SentimentIntensityAnalyzer()
    if isinstance(text, list):
        text = ' '.join(text)
    sentiment = sid.polarity_scores(text)
    return sentiment

df['sentiment'] = df['processed_text'].apply(sentiment_analysis)
df['emotions'] = df['processed_text'].apply(emotion_analysis)
df['sentiment'] = df['sentiment'].apply(lambda x: x['compound'])
df['emotions'] = df['emotions'].apply(lambda x: max(x, key=x.get))

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [82]:
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values(by='date')
df_agg = df.groupby('date')['sentiment'].mean().reset_index()

# Pie chart for 'emotions'
plt.figure(figsize=(5,5))
df['emotions'].value_counts().plot(kind='pie', autopct='%1.1f%%', explode=[0.1, 0, 0, 0, 0], shadow=True)
plt.title('Emotions Pie Chart')
plt.show() 

# Histogram for 'sentiment'
plt.figure(figsize=(5,5))
df['sentiment'].plot(kind='hist')
plt.title('Sentiment Histogram')
plt.show()

# Line chart for 'sentiment' over time
plt.figure(figsize=(20,5))
df_agg.plot(kind='line', x='date', y='sentiment')
plt.title('Average sentiment over time')
plt.show()



KeyError: 'Column not found: sentiment'

In [79]:
# Wyniki dla sentymentu
sentiment_results = df['sentiment'].apply(pd.Series)
print(sentiment_results.describe())

# Wyniki dla emocji
emotion_results = df['emotions'].apply(pd.Series)
print(emotion_results.describe())

                 0
count  1000.000000
mean      0.024150
std       0.405744
min      -0.953300
25%       0.000000
50%       0.000000
75%       0.211025
max       0.944200
            0
count    1000
unique      5
top     Happy
freq      401


In [80]:
df = pd.read_csv('tweets.csv')
df.shape

(1000, 6)

In [81]:
# df = df[~df['content'].str.contains('@@@@@@@@')]
# df = df[~df['content'].str.contains('http://')]
# df = df[~df['content'].str.contains('https://')]
# df = df.dropna()
# df.shape